<a href="https://colab.research.google.com/github/s0miya/Alzheimer-s-Medical-chatbot-using-LLM/blob/main/Alzheimer's_Medical_chatbot_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
from transformers import (TextDataset, DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments)
import pandas as pd
import matplotlib.pyplot as plt
import wandb
wandb.init(mode="disabled")
import warnings
warnings.filterwarnings('ignore')

In [ ]:

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Alzheimers Medical chatbot/ACD.zip', usecols=[1,2])
df.sample(5)

,Questions,Answers
17424,How does Alzheimer's impact emotions and mood?,Alzheimer's can significantly impact emotions ...
15107,How does memory loss impact Alzheimer's patients?,Memory loss is a hallmark symptom of Alzheimer...
368,Are there specific therapies or activities tha...,Absolutely. Cognitive stimulation therapy (CST...
4388,Can Vitamin D supplementation benefit individu...,The impact of Vitamin D supplementation on cha...
8306,How do head injuries contribute to the develop...,Head injuries may trigger pathological changes...


In [ ]:
!unzip ACD.zip

unzip:  cannot find or open ACD.zip, ACD.zip.zip or ACD.zip.ZIP.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18587 entries, 0 to 18586
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Questions  18587 non-null  object
 1   Answers    18584 non-null  object
dtypes: object(2)
memory usage: 290.5+ KB


In [ ]:
def load_dataset(file_path, tokenizer, block_size=1024):
  dataset_train=TextDataset(
  tokenizer=tokenizer,
  file_path=file_path,
  block_size=block_size
  )
  return dataset_train

In [ ]:
def load_data_collator(tokenizer, mlm=False):
  data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=mlm)
  return data_collator

In [ ]:
def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)
  tokenizer.save_pretrained(output_dir)
  model = GPT2LMHeadModel.from_pretrained(model_name)
  model.save_pretrained(output_dir)
  training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=overwrite_output_dir,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=per_device_train_batch_size,
      logging_dir="./logs",
      logging_steps=100,  # Log every 100 steps
      save_steps=500,  # Save checkpoint every 500 steps
      logging_first_step=True,
      save_total_limit=2,
      learning_rate=.0001
      )
  trainer = Trainer(model = model, args = training_args, data_collator = data_collator, train_dataset = train_dataset,)
  hist = trainer.train()
  trainer.save_model()
  return trainer,hist

In [ ]:
train_file_path='/content/drive/MyDrive/Colab Notebooks/Alzheimers Medical chatbot/train.csv'
model_name='gpt2'
output_dir='/content/drive/MyDrive/Colab Notebooks/Alzheimers Medical chatbot/custom_model'
overwrite_output_dir=True
per_device_eval_batch_size=2
num_train_epochs=10
per_device_train_batch_size = 2


In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [ ]:
model=GPT2LMHeadModel.from_pretrained(model_name)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model.base_model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
hist= train (
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,

)

Step,Training Loss
1,2.340900
100,1.722800


In [ ]:
print("global step:", hist[1].global_step)
print("epoch:", hist[1].metrics['epoch'])
print("train runtime:", hist[1].metrics['train_runtime'])
print("train samples per second"), hist[1].metrics['train_samples_per_second']
print("train steps per second"), hist[1].metrics['train_steps_per_second']
#print("train flops",hist[1].metrics['train_flops'])
print("train loss",hist[1].metrics['train_loss'])


In [ ]:
def load_model(model_path):
  model=GPT2LMHeadModel.from_pretrained(model_path)
  return model

In [ ]:
def load_tokenizer(tokenizer_path):
  tokenizer=GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

In [ ]:
def generate_text(model_path, sequence, max_length):
  model=load_model(model_path)
  ids=tokenizer.encode(f'{sequence}',return_tensors='pt')
  final_outputs=model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95,
  )
  return tokenizer.decode(final_outputs[0],skip_special_tokens=True)

In [ ]:
import os

In [ ]:
from huggingface_hub import login
login()



In [ ]:
model_path=os.mkdir('/content/custom_model', use_auth_token=True)
sequence=df['Questions'].iloc[100]
max_len=100
print("Q: ", df['Questions'].iloc[100])
print()
print("A: ", df['Answers'].iloc[100])
print()
print("G: ", generate_text('model_path_new', sequence, max_len,use_auth_token=True ))


In [ ]:
sequence=data['Question'].iloc[50]
max_len=200
print("Q: ", data['Questions'].iloc[50])
print()
print("A: ", data['Answers'].iloc[50])
print()
print("G: ", generate_text('model_path, sequence, max_length'))

In [ ]:
sequence=data['Question'].iloc[150]
max_len=200
print("Q: ", data['Questions'].iloc[150])
print()
print("A: ", data['Answers'].iloc[150])
print()
print("G: ", generate_text('model_path, sequence, max_length'))

In [ ]:
sequence="what is alzeimer's disease?"
max_len=500
print("Q: ", sequence)
print()
print("G: ", generate_text(model_path, sequence, max_length))


In [ ]:
'''def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs):
  tokenizer=GPT2Tokenizer.from_pretrained(model_name)
  train_dataset=load_dataset(train_file_path, tokenizer)
  data_collator=load_data_collator(tokenizer)
  max_steps=len(train_dataset)
  tokenizer.save_pretrained(output_dir)
  model=GPT2LMHeadModel.from_pretrained(model_name)
  model.save_pretrained(output_dir)
  training_args=TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=overwrite_output_dir,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=per_device_train_batch_size,
      logging_dir="./logs",
      logging_steps=100,
      save_steps=500,
      logging_first_step=True,
      save_total_limit=2,
      learning_rate=.0001,
      max_steps=int(1e6),
  )
  trainer=Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=train,max_steps=max_steps)
  hist=trainer.train()
  trainer.save_model()
  return trainer, hist'''